# Introduction

Seaborn is one of the most used libraries in Python for data visualization. I tried to create new meaningful datas from our data-set. Hoping will you enjoy this. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# First Contact With Data

In [ ]:
df1 = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_credits.csv")
df2 = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_movies.csv")

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df2.info()

# Calculating Incomes

In [ ]:
income = list(map(lambda a,b:a-b,list(df2["revenue"].values) ,list(df2["budget"].values)))
df2["income"] = income
df2.head(3)

In [ ]:
f, ax = plt.subplots(figsize = (16,16))
sns.heatmap(df2.corr(),ax = ax, annot = True, fmt = ".2f")

# Detecting Genres/Data Analyze

In [ ]:
dfGenres = df2[["genres","income"]]
gList1 = []
gList2 = []
for a in dfGenres["genres"]:
    tList = []
    for b in list(a.split("}, {")):
        c = b.split(" ")[3:]
        try:
            cLast2 = c[-1][-2:]
            if len(c) == 1:
                if cLast2 == "}]":
                    cLastCleaned = c[-1][:-2]
                    result = cLastCleaned
                else:
                    result = c[0]
            else:
                if cLast2 == "}]":
                    cLastCleaned = c[-1][:-2]
                    result = c[0] +" " + cLastCleaned
                else:
                    result = c[0] + " " + c[1]
        except IndexError:
            result = "None"
        tList.append(result)
    gList2 = gList2 + tList
    gList1.append(tList)

gList2 = list(dict.fromkeys(gList2))
dfGenres = dfGenres.drop(columns = ["genres"])
dfGenres["GenresBasic"] = gList1
dfGenres.head()

In [ ]:
meanList = []
for a in gList2:
    tempList = []
    for b in dfGenres.index:
        if a in str(dfGenres[dfGenres.index == b]["GenresBasic"]):
            tempList.append(int(dfGenres[dfGenres.index == b]["income"]))
        else:
            pass
    result2 = int(sum(tempList)/len(tempList))
    print(result2)
    meanList.append(result2)


# Avarage Incomes Accroding to Genres

In [ ]:
vData1 = pd.Series(dict(zip(gList2,meanList)))
f, ax =plt.subplots(figsize=(15,7))
sns.barplot(x = vData1.index[:17], y = vData1.values[:17], ax = ax)
plt.xticks(rotation = -90)
plt.ylabel("İncomes: Billion",color = "Blue",fontsize = 15)
plt.xlabel("Genres",color = "Blue",fontsize = 15)
plt.title("İncomes According to Genres",color = "Blue",fontsize = 15)
plt.grid()

# Detecting Companies:Data Analyze

In [ ]:
dfComp = df2[["production_companies","popularity","id","release_date","income","vote_count","vote_average"]]

In [ ]:
coList1 = []
coList2 = []
for a in dfComp["production_companies"]:
    tList2 = []
    for b in list(a.split("}, {")):
        try: 
            c = b.split(", ")[0].split(": ")[1]
            tList2.append(c)
        except IndexError:
            pass
    coList2 = coList2 + tList2
    coList1.append(tList2)
dfComp["BasicComp"]=coList1   

In [ ]:
from collections import Counter
coList3 = []
for each in Counter(coList2).most_common(10):
    each = str(each)
    each = each.split(", ")[0].split("('")[1]
    each = each[:-1]
    each = each[1:-1]
    coList3.append(each)
print(coList3)    

In [ ]:
import statistics
medList = []
for a in coList3:
    tempList = []
    for b in dfComp["id"]:
        if a in str(dfComp[dfComp["id"]==b]["BasicComp"]):
            tempList.append(int(dfComp[dfComp["id"]==b]["popularity"]))
        else:
            pass
    q = int(statistics.mean(tempList))
    print(q)
    medList.append(q)

In [ ]:
vData2 = pd.Series(dict(zip(coList3,medList)))


# Top 10 Company Popularity Rating

In [ ]:
f, ax =plt.subplots(figsize=(15,7))
sns.barplot(y = vData2.index, x = vData2.values, ax = ax)
plt.yticks(rotation = 0)
plt.ylabel(" Top 10 Companies",color = "Red",fontsize = 15)
plt.xlabel("Popularity Rate",color = "Red",fontsize = 15)
plt.title("Top 10 Company Popularity",color = "Red",fontsize = 15)
plt.grid()

In [ ]:
dfComp['release_date'] = pd.to_datetime(dfComp['release_date'])
dfComp.release_date

In [ ]:
dfComp.info()

In [ ]:
coList4 = []
for each in Counter(coList2).most_common(5):
    each = str(each)
    each = each.split(", ")[0].split("('")[1]
    each = each[:-1]
    each = each[1:-1]
    coList4.append(each)

print(coList4)    

# Top 5 Company's Incomes Year by Year

In [ ]:
colorList = ["Green","Blue","Orange","Brown","Purple"]
index = 0
for a in coList4:
    times = []
    incomes =[]
    for b in dfComp["id"]:
        if a in str(dfComp[dfComp["id"]==b]["BasicComp"]):
            times.append(int(str(dfComp[dfComp["id"]==b]["release_date"]).split("-")[0].split("  ")[1]))
            incomes.append(int(dfComp[dfComp["id"]==b]["income"]))
        else:
            pass
    vDataT = pd.Series(dict(zip(times,incomes)))
    f, ax =plt.subplots(figsize=(15,7))
    sns.pointplot(x = vDataT.index, y = vDataT.values, ax = ax, color = colorList[index])
    plt.xticks(rotation = -90)
    plt.ylabel("Incomes: Billion",color = colorList[index],fontsize = 15)
    plt.xlabel("Times",color = colorList[index],fontsize = 15)
    titleLabel = "The Incomes of "+a+" According To Years"
    plt.title(titleLabel,color = colorList[index],fontsize = 15)
    plt.grid()
    index += 1
    

# Correlation Between Number of Votes And Average of Votes

In [ ]:
sns.jointplot("vote_count","vote_average", data = dfComp,ratio = 3,color = "blue",kind ="reg", height = 7)
plt.title("Correlation Num Votes and Avrg",color = "orange",fontsize = 12)
plt.show()

In [ ]:
gVotes = []
gCompanies = []
for a in coList4:
    votes = []
    companies = []
    for b in dfComp["id"]:
        if a in str(dfComp[dfComp["id"]==b]["BasicComp"]):
            companies.append(a)
            votes.append(float(dfComp[dfComp["id"]==b]["vote_average"]))
        else:
            pass
    gVotes = gVotes + votes
    gCompanies = gCompanies + companies    

In [ ]:
dVC = {"Companies":gCompanies,"Votes":gVotes}
vData3 = pd.DataFrame(dVC)

# Detecting Outliers for Top 5 Companies

In [ ]:
f, ax =plt.subplots(figsize=(15,7))
sns.boxplot(data = vData3, x = "Companies", y = "Votes", ax = ax)
plt.ylabel("Votes",color = "r",fontsize = 15)
plt.xlabel("Companies",color = "red",fontsize = 15)
plt.xticks(color = "brown",fontsize = 10, rotation =0)
plt.title("The Votes of Top 5 Companies",color = "r",fontsize = 15)
plt.show()

# A Violin Graph For Votes

In [ ]:
f, ax =plt.subplots(figsize=(15,7))
sns.violinplot(data = vData3, y = "Votes", ax = ax, x = "Companies")
plt.legend()
plt.ylabel("Votes",color = "r",fontsize = 15)
plt.xlabel("Companies",color = "r",fontsize = 15)
plt.xticks(color = "blue",fontsize = 10, rotation =0)
plt.title("The Votes of Top 5 Companies",color = "r",fontsize = 15)
plt.show()

# Looking At Original Languages Except English

In [ ]:
f, ax =plt.subplots(figsize=(15,7))
sns.countplot(df2.original_language[df2["original_language"]!= "en"])
plt.legend()
plt.ylabel("Frequency",color = "r",fontsize = 15)
plt.xlabel("Languages",color = "r",fontsize = 15)
plt.xticks(color = "blue",fontsize = 10, rotation =0)
plt.title("Original Languages Except English",color = "r",fontsize = 15)
plt.show()